<a href="https://colab.research.google.com/github/Abhisekh97/RankNet_tf_keras/blob/main/RankNet_tensorflow_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, activations, losses, Model, Input
from tensorflow.nn import leaky_relu
import numpy as np
from itertools import combinations
from tensorflow.keras.utils import plot_model, Progbar
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


##  Model Architecture

In [8]:
class RankNet(Model):
  def __init__(self):
    super().__init__(self)
    self.dense = [layers.Dense(16, activation=leaky_relu), layers.Dense(8, activation=leaky_relu)]
    self.o = layers.Dense(1, activation='linear')
    self.oi_minus_oj = layers.Subtract()


  def call(self, input):
    xi , xj = input
    densei = self.dense[0](xi)
    densej = self.dense[0](xj)
    for dense in self.dense[1:]:
      densei = dense(densei)
      densej = dense(densej)

    oi = self.o(densei)
    oj = self.o(densej)
    oij = self.oi_minus_oj([oi, oj])
    output = layers.Activation('sigmoid')(oij)
    return output

  def build_graph(self):
    x = [Input(shape=(10)), Input(shape=(10))]
    return Model(inputs=x, outputs=self.call(x))

model = RankNet()
model.build_graph().summary()


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 input_8 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 dense_9 (Dense)             (None, 16)                   176       ['input_7[0][0]',             
                                                                     'input_8[0][0]']             
                                                                                                  
 dense_10 (Dense)            (None, 8)                    136       ['dense_9[0][0]',       